In [1]:
import polars as pl
pl.Config.set_tbl_rows(1000)  # or whatever number of rows you want to see
import pathlib
import os
import psutil
import gc

In [ ]:
def clear_memory(df_to_remove=None):
    """Clear memory and print memory usage statistics"""
    try:
        # Print initial state
        print("\nInitial memory state:")
        process = psutil.Process(os.getpid())
        initial_memory_mb = process.memory_info().rss / 1024 / 1024
        print(f"Current Memory Usage: {initial_memory_mb:.2f} MB ({initial_memory_mb/1024:.2f} GB)")
        
        # Remove specific DataFrame if provided
        if df_to_remove is not None:
            if isinstance(df_to_remove, list):
                # If a list of DataFrames is provided
                for df in df_to_remove:
                    del df
            else:
                # If a single DataFrame is provided
                del df_to_remove
        
        # Force garbage collection
        gc.collect()
        
        # Get new memory info
        new_memory_mb = process.memory_info().rss / 1024 / 1024
        memory_freed = initial_memory_mb - new_memory_mb
        
        print(f"\nCurrent Memory Usage: {new_memory_mb:.2f} MB ({new_memory_mb/1024:.2f} GB)")
        print(f"Available System Memory: {psutil.virtual_memory().available / 1024 / 1024 / 1024:.2f} GB")
        print(f"Memory Utilization: {psutil.virtual_memory().percent}%")
        
        if memory_freed > 0:
            print(f"Memory freed: {memory_freed:.2f} MB")
            
    except Exception as e:
        print(f"Memory cleanup failed: {e}")


def main():
    """Main function to load and process the data"""
    try:
        print("Initial memory state:")
        clear_memory()
        
        print("\nLoading CSV file...")
        # Read CSV with updated schema_overrides parameter
        master_df = pl.read_csv('dataset/original_dataset/kcc_dataset.csv',
            schema_overrides={
                'Year': pl.Int32,
                'Month': pl.Int32,
                'Day': pl.Int32,
                'Crop': pl.Utf8,
                'DistrictName': pl.Utf8,
                'QueryType': pl.Utf8,
                'Season': pl.Utf8,
                'Sector': pl.Utf8,
                'StateName': pl.Utf8,
                'QueryText': pl.Utf8,
                'KccAns': pl.Utf8,
                'Category': pl.Utf8,
                'BlockName': pl.Utf8
            },
            low_memory=True
        ).drop(['BlockName', 'Category'])
        
        print("\nAfter loading CSV:")
        clear_memory()
        
        return master_df
        
    except Exception as e:
        print(f"Error in data processing: {e}")
        return None


# First clear any existing DataFrames
clear_memory()

### Working on India level data

In [5]:
# Usage examples:
# Load the data
master_df = main()

Initial memory state:

Initial memory state:
Current Memory Usage: 82.71 MB (0.08 GB)

Current Memory Usage: 82.71 MB (0.08 GB)
Available System Memory: 243.70 GB
Memory Utilization: 2.9%

Loading CSV file...

After loading CSV:

Initial memory state:
Current Memory Usage: 11777.02 MB (11.50 GB)

Current Memory Usage: 11777.02 MB (11.50 GB)
Available System Memory: 232.26 GB
Memory Utilization: 7.4%


In [6]:
master_df.head()

Year,Month,Day,Crop,DistrictName,QueryType,Season,Sector,StateName,QueryText,KccAns
i32,i32,i32,str,str,str,str,str,str,str,str
2006,1,17,"""1275""","""SAGAR""","""99""","""RABI""","""HORTICULTURE""","""MADHYA PRADESH""","""how to control flower drop in …","""spray planofix4mlpump"""
2006,1,17,"""964""","""SAGAR""","""Disease Management""","""RABI""","""ANIMAL HUSBANDRY""","""MADHYA PRADESH""","""how tyo control diseases in bu…",null
2006,1,17,"""1279""","""SAGAR""","""76""","""RABI""","""HORTICULTURE""","""MADHYA PRADESH""","""how to control fruit borer in …","""should be spray profenophos 35…"
2006,1,17,"""1064""","""SAGAR""","""3""","""RABI""","""AGRICULTURE""","""MADHYA PRADESH""","""how to control of yellow moisa…","""should be spray metasystox 35m…"
2006,1,17,"""1279""","""DAMOH""","""76""","""RABI""","""HORTICULTURE""","""MADHYA PRADESH""","""how to control white fly in br…","""should be spray metasystox 35m…"


In [7]:
master_df.shape

(41987874, 11)

#### Filtering: Null Analysis

In [8]:
total_rows = master_df.shape[0]

# Create a DataFrame with both counts and percentages
null_analysis = (
    pl.DataFrame({
        'Column': master_df.columns,
        'Total_Values': total_rows,
        'Null_Count': master_df.null_count().row(0),
    })
    .with_columns([
        (pl.col('Null_Count') / pl.col('Total_Values') * 100).round(2).alias('Null_Percentage')
    ])
    .sort('Null_Percentage', descending=True)  # Sort by percentage descending
)

print("\nNull value analysis:")
print(null_analysis)


Null value analysis:
shape: (11, 4)
┌──────────────┬──────────────┬────────────┬─────────────────┐
│ Column       ┆ Total_Values ┆ Null_Count ┆ Null_Percentage │
│ ---          ┆ ---          ┆ ---        ┆ ---             │
│ str          ┆ i32          ┆ i64        ┆ f64             │
╞══════════════╪══════════════╪════════════╪═════════════════╡
│ Season       ┆ 41987874     ┆ 26665089   ┆ 63.51           │
│ KccAns       ┆ 41987874     ┆ 4320412    ┆ 10.29           │
│ QueryType    ┆ 41987874     ┆ 1333503    ┆ 3.18            │
│ Crop         ┆ 41987874     ┆ 172930     ┆ 0.41            │
│ Sector       ┆ 41987874     ┆ 85083      ┆ 0.2             │
│ QueryText    ┆ 41987874     ┆ 10560      ┆ 0.03            │
│ Year         ┆ 41987874     ┆ 0          ┆ 0.0             │
│ Month        ┆ 41987874     ┆ 0          ┆ 0.0             │
│ Day          ┆ 41987874     ┆ 0          ┆ 0.0             │
│ DistrictName ┆ 41987874     ┆ 0          ┆ 0.0             │
│ StateName    ┆ 4

In [9]:
# First check the initial shape
print(f"Original shape: {master_df.shape}")

# List the specific columns you want to check for nulls
columns_to_check = [
    'KccAns',
    'QueryType', 
    'Crop',
    'Sector',
    'QueryText']  # replace with your column names

# Drop nulls only from specified columns
master_df = master_df.drop_nulls(subset=columns_to_check)

# Print new shape to see how many rows were removed
print(f"Shape after removing nulls: {master_df.shape}")

# # Optional: Check if there are any nulls remaining in these columns
# null_check = master_df.select(columns_to_check).null_count()
# print("\nNull counts in specified columns after cleaning:")
# print(null_check)

Original shape: (41987874, 11)
Shape after removing nulls: (36267631, 11)


In [10]:
# Replace nulls in Season column with "Unspecified"
master_df = master_df.with_columns(
    pl.col('Season').fill_null(value="Unspecified")
)

# Verify the replacement
print(master_df['Season'].unique())


shape: (5,)
Series: 'Season' [str]
[
	"0"
	"RABI"
	"Unspecified"
	"KHARIF"
	"JAYAD"
]


#### Filtering: ONLY `digit` values in columns that are supposed to have texts

In [11]:
def check_digit_only_percentages(df):
    total_rows = df.shape[0]
    results = []
    
    for col in df.columns:
        # Count values that contain only digits using regex
        digit_only_count = df.filter(
            pl.col(col).cast(pl.Utf8).str.contains(r'^\d+$')
        ).height
        
        # Calculate percentage
        percentage = (digit_only_count / total_rows * 100)
        
        results.append({
            'Column': col,
            'Total_Values': total_rows,
            'Digit_Only_Count': digit_only_count,
            'Percentage': round(percentage, 2)
        })
    
    # Convert results to Polars DataFrame and sort by percentage
    return (pl.DataFrame(results)
            .sort('Percentage', descending=True))

# Run the analysis
digit_analysis = check_digit_only_percentages(master_df)

# Display results
print("\nPercentage of digit-only values in each column:")
print(digit_analysis)


Percentage of digit-only values in each column:
shape: (11, 4)
┌──────────────┬──────────────┬──────────────────┬────────────┐
│ Column       ┆ Total_Values ┆ Digit_Only_Count ┆ Percentage │
│ ---          ┆ ---          ┆ ---              ┆ ---        │
│ str          ┆ i64          ┆ i64              ┆ f64        │
╞══════════════╪══════════════╪══════════════════╪════════════╡
│ Year         ┆ 36267631     ┆ 36267631         ┆ 100.0      │
│ Month        ┆ 36267631     ┆ 36267631         ┆ 100.0      │
│ Day          ┆ 36267631     ┆ 36267631         ┆ 100.0      │
│ QueryType    ┆ 36267631     ┆ 2785401          ┆ 7.68       │
│ Crop         ┆ 36267631     ┆ 1433408          ┆ 3.95       │
│ Season       ┆ 36267631     ┆ 535174           ┆ 1.48       │
│ Sector       ┆ 36267631     ┆ 298539           ┆ 0.82       │
│ KccAns       ┆ 36267631     ┆ 41759            ┆ 0.12       │
│ QueryText    ┆ 36267631     ┆ 2229             ┆ 0.01       │
│ DistrictName ┆ 36267631     ┆ 0       

In [12]:
# List of columns to check
columns_to_check = [
    'QueryType', 
    'Crop',
    'Season', 
    'Sector', 
    'KccAns',
    'QueryText'
    ]

# First print original shape
# print(f"Original shape: {master_df.shape}")

# Create filter condition to remove rows with only digits in specified columns
master_df = master_df.filter(
    ~(pl.col(columns_to_check[0]).cast(pl.Utf8).str.contains(r'^\d+$')) &
    ~(pl.col(columns_to_check[1]).cast(pl.Utf8).str.contains(r'^\d+$')) &
    ~(pl.col(columns_to_check[2]).cast(pl.Utf8).str.contains(r'^\d+$')) &
    ~(pl.col(columns_to_check[3]).cast(pl.Utf8).str.contains(r'^\d+$')) &
    ~(pl.col(columns_to_check[4]).cast(pl.Utf8).str.contains(r'^\d+$')) &
    ~(pl.col(columns_to_check[5]).cast(pl.Utf8).str.contains(r'^\d+$'))
)

# Print new shape
print(f"New shape: {master_df.shape}")

# Print how many rows were removed
# rows_removed = master_df.shape[0] - filtered_master_df.shape[0]
# print(f"Rows removed: {rows_removed}")

# Optional: Verify by checking digit-only percentages in new DataFrame
def check_digit_only_percentages(df, cols):
    total_rows = df.shape[0]
    results = []
    
    for col in cols:
        digit_only_count = df.filter(
            pl.col(col).cast(pl.Utf8).str.contains(r'^\d+$')
        ).height
        
        percentage = (digit_only_count / total_rows * 100)
        
        results.append({
            'Column': col,
            'Total_Values': total_rows,
            'Digit_Only_Count': digit_only_count,
            'Percentage': round(percentage, 2)
        })
    
    return pl.DataFrame(results).sort('Percentage', descending=True)

# Check percentages in new DataFrame
# print("\nPercentage of digit-only values in each column after filtering:")
# print(check_digit_only_percentages(master_df, columns_to_check))

# Optionally clear old DataFrame from memory
# clear_memory(df_to_remove=master_df)

New shape: (32724854, 11)


In [13]:
master_df = master_df.with_columns([
    pl.concat_str([
        pl.col('Day').cast(pl.Utf8).str.zfill(2),
        pl.col('Month').cast(pl.Utf8).str.zfill(2),
        pl.col('Year').cast(pl.Utf8)
    ], separator='-')
    .str.strptime(pl.Datetime, format='%d-%m-%Y')
    .alias('Date')
])

In [14]:
master_df.head()

Year,Month,Day,Crop,DistrictName,QueryType,Season,Sector,StateName,QueryText,KccAns,Date
i32,i32,i32,str,str,str,str,str,str,str,str,datetime[μs]
2006,1,17,"""BovineCowBuffalo""","""INDORE""","""Dairy Production""","""RABI""","""ANIMAL HUSBANDRY""","""MADHYA PRADESH""","""How to increase milk productio…","""you can give gylox powder 100 …",2006-01-17 00:00:00
2007,1,5,"""Coconut""","""SAMASTIPUR""","""Fertilizer Use and Availabilit…","""KHARIF""","""HORTICULTURE""","""BIHAR""","""FERTILIZER DOSES OF COCONUT""","""FERTILIZER ARE NPK 1:2:2 KGPL…",2007-01-05 00:00:00
2007,1,5,"""Others""","""KHAGARIA""","""Weather""","""KHARIF""","""AGRICULTURE""","""BIHAR""","""WEATHER CONDATION IN KHAGARIA""","""ANSWER GIVEN TO DETAIL """,2007-01-05 00:00:00
2007,1,20,"""Others""","""KATIHAR""","""Weather""","""Unspecified""","""HORTICULTURE""","""BIHAR""","""Weather condition of Katihar d…","""Answer given to farmer by Inte…",2007-01-20 00:00:00
2007,1,1,"""Onion""","""AHMADNAGAR""","""Agriculture Mechanization""","""Unspecified""","""HORTICULTURE""","""MAHARASHTRA""","""blight on onion""","""copper oxycloride25ml10lit of …",2007-01-01 00:00:00


In [17]:
master_df.write_csv(
    "dataset/processed_dataset/India_level_data_filtered.csv",
    separator=",",  # default is comma
    include_header=True,  # include column names
    null_value="",  # how to represent null values
    float_precision=3  # decimal places for float values
)

In [22]:
! pip install pyarrow

In [24]:
all_India_QueryType = (master_df
    .select(pl.col('QueryType'))
    .group_by('QueryType')
    .agg(pl.count('QueryType').alias('count'))
    .with_columns([
        (pl.col('count') / pl.col('count').sum() * 100).round(2).alias('percentage')
    ])
    .sort('count', descending=True)
)

import plotly.graph_objects as go

# Get top 10 rows
top_10 = all_India_QueryType.head(10).to_pandas()

# Calculate the sum of percentages for remaining rows (Others)
others_percentage = all_India_QueryType.slice(10).select('percentage').sum().item()

# Create labels and values for the chart
labels = list(top_10['QueryType']) + ['Others']
values = list(top_10['percentage']) + [others_percentage]

# Create the donut chart
fig = go.Figure(data=[go.Pie(
    labels=labels,
    values=values,
    hole=0.4,
    textinfo='label+percent',
    textposition='outside',
    showlegend=False,
    direction='clockwise',
    sort=False
)])

# Update layout
fig.update_layout(
    title={
        'text': 'Distribution of Query Types (Top 10)',
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    width=1200,
    height=800,
    font=dict(size=14)
)

fig.show()

ModuleNotFoundError: pa.Table requires 'pyarrow' module to be installed

In [ ]:
master_df.head()

In [ ]:
# Calculate counts and percentages by StateName and QueryType
state_querytype_distribution = (master_df
    .group_by(['StateName', 'QueryType'])
    .agg(pl.count().alias('count'))
    .with_columns([
        pl.col('count').sum().over('StateName').alias('state_total')
    ])
    .with_columns([
        (pl.col('count') / pl.col('state_total') * 100).round(2).alias('percentage')
    ])
    .sort(['StateName', 'count'], descending=[False, True])
)

# Display results
print("\nQueryType Distribution by State:")
print(state_querytype_distribution)

# Optional: To see distribution for a specific state
state_name = "MAHARASHTRA"  # replace with state you want to see
state_distribution = state_querytype_distribution.filter(pl.col('StateName') == state_name)
print(f"\nQueryType Distribution for {state_name}:")
print(state_distribution)

In [ ]:
query_type_stats = (master_df
    .group_by('QueryType')
    .agg(pl.count().alias('count'))
    .with_columns([
        (pl.col('count') / pl.col('count').sum() * 100).round(2).alias('percentage'),
        pl.col('count').sum().alias('total')
    ])
    .sort('count', descending=True)
)

print("\nQueryType Distribution:")
print(query_type_stats)

In [ ]:
columns_to_check = [
    'Crop', 
    'DistrictName',
    'QueryType', 
    'Season',
    'Sector',
    'StateName']
conditions = [~pl.col(col).str.contains(r'\d') for col in columns_to_check]
master_df = master_df.filter(pl.all_horizontal(conditions))

print(master_df.shape)

In [ ]:
(41987874 - 11553157)/41987874

In [ ]:
master_df = master_df.filter(~pl.col('QueryType').str.contains(r'\d'))
print(master_df.shape)


#### Filtering: removing all numeric values from Crop

In [ ]:
master_df = master_df.filter(~pl.col('Crop').str.contains(r'\d'))
print(master_df.shape)

#### Filtering: removing all numeric values from Sector

In [ ]:
master_df = master_df.filter(~pl.col('Sector').str.contains(r'\d'))
print(master_df.shape)

In [ ]:
master_df['StateName'].unique()

In [ ]:
# Using filter() method for Polars DataFrame
maharashtra_df = master_df.filter(pl.col('StateName') == 'MAHARASHTRA')

# Print shape to verify
print(f"Shape of maharashtra_df: {maharashtra_df.shape}")

In [ ]:
clear_memory(df_to_remove=master_df)

In [ ]:
# FILTER CONDITION: Filter master_df to keep only QueryType with string values
maharashtra_df_stringQT = maharashtra_df.filter(~pl.col('QueryType').str.contains(r'\d'))

print(maharashtra_df_stringQT.shape)

In [ ]:
clear_memory(df_to_remove= maharashtra_df)

In [ ]:
maharashtra_df_stringCrop = maharashtra_df_stringQT.filter(~pl.col('Crop').str.contains(r'\d'))

print(maharashtra_df_stringCrop.shape)

In [ ]:
clear_memory(df_to_remove= maharashtra_df_stringQT)

In [ ]:
maharashtra_df_stringCrop.head()

In [ ]:
type_of_query = list(master_df['QueryType'].unique())

# First remove None values
valid_queries = [query for query in type_of_query if query is not None]

# Then search for fertilizer/fertiliser
fertilizer_queries = [query for query in valid_queries 
                     if 'fertilizer' in str(query).lower() or 'fertiliser' in str(query).lower()]

# Print the matches
print("Queries related to fertilizer/fertiliser:")
for query in fertilizer_queries:
    print(f"- {query}")

In [ ]:
# Use .filter() with pl.col('QueryType').is_in() to subset the DataFrame
fertilizer_df = master_df.filter(pl.col('QueryType').is_in(fertilizer_queries))

# Print the shape to verify
print(f"Shape of fertilizer_df: {fertilizer_df.shape}")

# Optional: Display unique QueryTypes in the filtered DataFrame to verify
print("\nUnique QueryTypes in fertilizer_df:")
print(fertilizer_df['QueryType'].unique())

In [ ]:
# FILTER CONDITION: Filter master_df to keep only QueryType with string values
master_df_filtered_QueryType = master_df.filter(~pl.col('QueryType').str.contains(r'\d'))
clear_memory(df_to_remove=master_df)  # Remove the original large DataFrame

print(master_df_filtered_QueryType.shape)

In [ ]:
# FILTER CONDITION: Filter master_df to keep only QueryType with string values
master_df_filtered_Crop = master_df_filtered_QueryType.filter(~pl.col('Crop').str.contains(r'\d'))

print(master_df_filtered_Crop.shape)

In [ ]:
clear_memory(df_to_remove=master_df)

In [ ]:
master_df_filtered_QueryType.head()

In [ ]:
master_df_filtered_QueryType['Crop'].unique()

In [ ]:
result = (master_df_filtered_QueryType
    .select(pl.col('QueryType'))
    .group_by('QueryType')
    .agg(pl.count('QueryType').alias('count'))
    .with_columns([
        (pl.col('count') / pl.col('count').sum() * 100).round(2).alias('percentage')
    ])
    .sort('count', descending=True)
)

In [ ]:
def clear_memory():
    """Clear memory and print memory usage statistics"""
    try:
        # Force garbage collection
        gc.collect()
        
        # Get current process
        process = psutil.Process(os.getpid())
        
        # Get memory info
        memory_mb = process.memory_info().rss / 1024 / 1024
        
        # Print memory info with more context
        print(f"\nCurrent Memory Usage: {memory_mb:.2f} MB ({memory_mb/1024:.2f} GB)")
        print(f"Available System Memory: {psutil.virtual_memory().available / 1024 / 1024 / 1024:.2f} GB")
        print(f"Memory Utilization: {psutil.virtual_memory().percent}%")
        
        # Force garbage collection again
        gc.collect()
        
        # Get new memory info
        new_memory_mb = process.memory_info().rss / 1024 / 1024
        memory_freed = memory_mb - new_memory_mb
        
        if memory_freed > 0:
            print(f"Memory freed by garbage collection: {memory_freed:.2f} MB")
            
    except Exception as e:
        print(f"Memory cleanup failed: {e}")

In [ ]:
# Verify the results
result = (master_df_filtered_QueryType
    .select(pl.col('QueryType'))
    .group_by('QueryType')
    .agg(pl.count('QueryType').alias('count'))
    .with_columns([
        (pl.col('count') / pl.col('count').sum() * 100).round(2).alias('percentage')
    ])
    .sort('count', descending=True)
)

In [ ]:
import plotly.graph_objects as go

# Get top 10 rows
top_10 = master_df_filtered_QueryType.head(10).to_pandas()

# # Calculate the sum of percentages for remaining rows (Others)
# others_percentage = master_df_filtered_QueryType.slice(10).select('percentage').sum().item()

# # Create labels and values for the chart
# labels = list(top_10['QueryType']) + ['Others']
# values = list(top_10['percentage']) + [others_percentage]

# # Create the donut chart
# fig = go.Figure(data=[go.Pie(
#     labels=labels,
#     values=values,
#     hole=0.4,
#     textinfo='label+percent',
#     textposition='outside',  # Changed from 'inside' to 'outside'
#     showlegend=False,  # Changed to False to remove the legend
#     direction='clockwise',
#     sort=False
# )])

# # Update layout
# fig.update_layout(
#     title={
#         'text': 'What Indian Farmers Query on',
#         'y':0.95,
#         'x':0.5,
#         'xanchor': 'center',
#         'yanchor': 'top'
#     },
#     width=1200,
#     height=800,
#     font=dict(size=14)
# )

# fig.show()

In [ ]:
top_10

In [ ]:
master_df_filtered_QueryType.head()

In [ ]:
((master_df.shape[0] - master_df_filtered_QueryType.shape[0])/(master_df.shape[0]))*199

In [ ]:
master_df.head()

In [ ]:
all_India_QueryType = (master_df
    .select(pl.col('QueryType'))
    # Add a filter to exclude QueryType containing numbers
    .filter(~pl.col('QueryType').str.contains(r'\d'))
    .group_by('QueryType')
    .agg(pl.count('QueryType').alias('count'))
    .with_columns([
        (pl.col('count') / pl.col('count').sum() * 100).round(2).alias('percentage')
    ])
    .sort('count', descending=True)
)


import plotly.graph_objects as go

# Get top 10 rows
top_10 = all_India_QueryType.head(10).to_pandas()

# Calculate the sum of percentages for remaining rows (Others)
others_percentage = all_India_QueryType.slice(10).select('percentage').sum().item()

# Create labels and values for the chart
labels = list(top_10['QueryType']) + ['Others']
values = list(top_10['percentage']) + [others_percentage]

# Create the donut chart
fig = go.Figure(data=[go.Pie(
    labels=labels,
    values=values,
    hole=0.4,
    textinfo='label+percent',
    textposition='outside',  # Changed from 'inside' to 'outside'
    showlegend=False,  # Changed to False to remove the legend
    direction='clockwise',
    sort=False
)])

# Update layout
fig.update_layout(
    title={
        'text': 'Distribution of Query Types (Top 10)',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    width=1200,
    height=800,
    font=dict(size=14)
)

fig.show()

In [ ]:
values

In [ ]:
import plotly.graph_objects as go

# Get top 10 rows
top_10 = all_India_QueryType.head(10).to_pandas()

# Calculate the sum of percentages for remaining rows (Others)
others_percentage = all_India_QueryType.slice(10).select('percentage').sum().item()

# Create labels and values for the chart - multiply values by 100
labels = list(top_10['QueryType']) + ['Others']
values = [x * 100 for x in list(top_10['percentage'])] + [others_percentage * 100]

# Create the donut chart
fig = go.Figure(data=[go.Pie(
    labels=labels, 
    values=values,
    hole=0.4,
    textinfo='label+percent',
    textposition='inside',
    texttemplate='%{label}<br>%{percent:.1f}%',
    showlegend=True,
    direction='clockwise',
    sort=False,
    pull=[0.1] + [0] * len(labels[1:])
)])

# Update layout
fig.update_layout(
    title={
        'text': 'Distribution of Query Types (Top 10)',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    width=900,
    height=700,
    font=dict(size=12),
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=-0.1,
        xanchor="center",
        x=0.5
    )
)

fig.show()

In [ ]:
all_India_QueryType

In [ ]:
#!/usr/bin/env python3

# Import all required libraries
import polars as pl
import pathlib
import os
import psutil
import gc

def clear_memory():
    """Clear memory and print memory usage statistics"""
    try:
        # Force garbage collection
        gc.collect()
        
        # Get current process
        process = psutil.Process(os.getpid())
        
        # Get memory info
        memory_mb = process.memory_info().rss / 1024 / 1024
        
        # Print memory info with more context
        print(f"\nCurrent Memory Usage: {memory_mb:.2f} MB ({memory_mb/1024:.2f} GB)")
        print(f"Available System Memory: {psutil.virtual_memory().available / 1024 / 1024 / 1024:.2f} GB")
        print(f"Memory Utilization: {psutil.virtual_memory().percent}%")
            
        # Force garbage collection again
        gc.collect()
        
        # Get new memory info
        new_memory_mb = process.memory_info().rss / 1024 / 1024
        memory_freed = memory_mb - new_memory_mb
        
        if memory_freed > 0:
            print(f"Memory freed by garbage collection: {memory_freed:.2f} MB")
        
    except Exception as e:
        print(f"Memory cleanup failed: {e}")

def main():
    """Main function to load and process the data"""
    try:
        print("Initial memory state:")
        clear_memory()
        
        print("\nLoading CSV file...")
        # Read CSV with updated schema_overrides parameter
        master_df = pl.read_csv('dataset/original_dataset/kcc_dataset.csv',
            schema_overrides={
                'Year': pl.Int32,
                'Month': pl.Int32,
                'Day': pl.Int32,
                'Crop': pl.Utf8,
                'DistrictName': pl.Utf8,
                'QueryType': pl.Utf8,
                'Season': pl.Utf8,
                'Sector': pl.Utf8,
                'StateName': pl.Utf8,
                'QueryText': pl.Utf8,
                'KccAns': pl.Utf8,
                'Category': pl.Utf8,
                'BlockName': pl.Utf8
            },
            low_memory=True
        ).drop(['BlockName', 'Category'])
        
        print("\nAfter loading CSV:")
        clear_memory()
        
        return master_df
        
    except Exception as e:
        print(f"Error in data processing: {e}")
        return None

if __name__ == "__main__":
    result_df = main()

In [ ]:
master_df.head()

In [ ]:
master_df['QueryType']].value_counts()

In [ ]:
master_df_cropInsurance.head()

In [ ]:
import polars as pl
import plotly.express as px
import plotly.graph_objects as go

# Create a date column using pl.date
master_df_cropInsurance = master_df_cropInsurance.with_columns([
    pl.date(
        year=pl.col('Year'),
        month=pl.col('Month'),
        day=1
    ).alias('Date')
])

# Group by date and state, count occurrences
monthly_state_counts = (
    master_df_cropInsurance
    .group_by(['Date', 'StateName'])
    .agg(
        pl.count().alias('count')
    )
    .sort('Date')
)

# Convert to pandas for easier plotting with plotly
monthly_state_df = monthly_state_counts.to_pandas()

# Create line plot
fig = px.line(
    monthly_state_df,
    x='Date',
    y='count',
    color='StateName',
    title='Crop Insurance Queries by State Over Time',
    labels={
        'Date': 'Month-Year',
        'count': 'Number of Queries',
        'StateName': 'State'
    }
)

# Customize layout
fig.update_layout(
    xaxis_title="Month-Year",
    yaxis_title="Number of Queries",
    legend_title="State",
    hovermode='x unified',
    template='plotly_white',
    # Improve readability
    xaxis=dict(
        tickangle=45,
        tickformat='%b %Y'
    ),
    # Add some margins for better display
    margin=dict(t=50, b=100)
)

# Add hover data
fig.update_traces(
    hovertemplate='<b>%{y}</b> queries<br>%{x|%B %Y}<extra></extra>'
)

# Show the plot
fig.show()

# Optional: Save the plot
# fig.write_html("crop_insurance_queries.html")

In [ ]:
wb_no_weather = master_df.filter(
    (pl.col('QueryType') != 'Weather') & 
    (~pl.col('QueryType').str.contains(r'^[0-9]+$'))
)

# Verify the results
result = (master_df
    .select(pl.col('QueryType'))
    .group_by('QueryType')
    .agg(pl.count('QueryType').alias('count'))
    .with_columns([
        (pl.col('count') / pl.col('count').sum() * 100).round(2).alias('percentage')
    ])
    .sort('count', descending=True)
)

In [ ]:
master_df['QueryType'].value_counts()

In [ ]:
state_list = list(master_df['StateName'].unique())

print(state_list)

In [ ]:
wb = master_df.filter(master_df['StateName'] == 'WEST BENGAL')

In [ ]:
wb.shape

In [ ]:
wb.head()

In [ ]:
import polars as pl
pl.Config.set_tbl_rows(100)  # or whatever number of rows you want to see

# First, let's create a filter that identifies if a string is numeric
wb_no_weather = master_df.filter(
    (pl.col('QueryType') != 'Weather') & 
    (~pl.col('QueryType').str.contains(r'^[0-9]+$'))
)

# Verify the results
result = (wb_no_weather
    .select(pl.col('QueryType'))
    .group_by('QueryType')
    .agg(pl.count('QueryType').alias('count'))
    .with_columns([
        (pl.col('count') / pl.col('count').sum() * 100).round(2).alias('percentage')
    ])
    .sort('count', descending=True)
)

In [ ]:
result

In [ ]:
import polars as pl
pl.Config.set_tbl_rows(100)  # or whatever number of rows you want to see

# Then run your query
result = (wb_no_weather
 .select(pl.col('QueryType'))
 .group_by('QueryType')
 .agg(pl.count('QueryType').alias('count'))
 .with_columns([
     (pl.col('count') / pl.col('count').sum() * 100).round(2).alias('percentage')
 ])
 .sort('count', descending=True))

result  # Display the result

In [ ]:
list(result['QueryType'])